In [1]:
import findspark
findspark.init('/opt/spark')  # Adjust the path if Spark is installed elsewhere

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("example").getOrCreate()

24/06/11 10:24:08 WARN Utils: Your hostname, chuot-HP-Pavilion-Laptop-14-ce3xxx resolves to a loopback address: 127.0.1.1; using 192.168.1.14 instead (on interface wlo1)
24/06/11 10:24:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/11 10:24:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Define the path to your JSON file
from pyspark.sql.functions import col, udf, to_date, when, regexp_extract, explode, col, regexp_replace, to_timestamp, concat, lit, trim
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType, StringType, ArrayType
import json
import re
import pymongo
json_file_path = 'careerviet.json'
# Read the JSON file into a DataFrame
df = spark.read.option("multiline","true").json(json_file_path)

# df_exploded = df.select(explode(col("location")).alias("unique_category"))

# unique_categories = df_exploded.select("unique_category").distinct().collect()
# print(len(unique_categories))
# for row in unique_categories:
#     pass
#     print(row.unique_category)
# unique_experience_df = df.select("location").distinct()

# unique_experience_df.show()
# from pyspark.sql.functions import col
unique_values = df.select(col("category")).distinct().collect()

# In ra các giá trị duy nhất
for row in unique_values:
    if row.category is None:
        print("None ne")
    print(row.category)


# print(df.size)


#rdd.toDF().select("salary").show(1)

# Salary


AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `category` cannot be resolved. Did you mean one of the following? [`age`, `logo`, `location`, `salary`, `sex`].;
'Project ['category]
+- Relation [age#0,benefit#1,certificate#2,company#3,description#4,experience#5,expiration#6,location#7,logo#8,province#9,requirement#10,role#11,salary#12,sex#13,title#14,type#15,update_time#16,url#17] json


In [11]:
df = spark.read.option("multiline","true").json(json_file_path)
# Salary
def parse_salary(salary):
    vnd_per_usd = 25
    if salary is None:
        return {"type": 0 }
    elif (salary.strip().lower() == "cạnh tranh"):
        return {"type": 3 }
    elif "VND" in salary:
            salary = salary.replace(" VND", "").replace(" Tr", "").replace(",", ".").strip()
            if "-" in salary:
                min_salary, max_salary = salary.split("-")
                return {"type": 1, "min": float(min_salary), "max": float(max_salary) }
            elif "Trên" in salary:
                min_salary = salary.replace("Trên", "").strip()
                return {"type": 5, "min": float(min_salary)}
            elif "Lên đến" in salary:
                max_salary = salary.replace("Lên đến", "").strip()
                return {"type": 4, "max": float(max_salary)}
    elif "USD" in salary:
        salary = salary.replace(" USD", "").replace(" Tr", "").replace(",", "").strip()
        if "-" in salary:
            min_salary, max_salary = salary.split("-")
            return {"type": 1, "min": float(min_salary)* vnd_per_usd, "max": float(max_salary)* vnd_per_usd}
        elif "Trên" in salary:
            min_salary = salary.replace("Trên", "").strip()
            return {"type": 5, "min": float(min_salary)* vnd_per_usd}
        elif "Lên đến" in salary:
            max_salary = salary.replace("Lên đến", "").strip()
            return {"type": 4, "min": float(max_salary)* vnd_per_usd}
    else:
        return {"type": 0 }
                
# Age
def parse_age(age):
    if age is None or age == "Không giới hạn tuổi":
        return { "type": 0 }
    # age = age.strip().lower()
    if "-" in age:
        strip_age = age.strip()
        min_age, max_age = age.strip().split("-")
        return { "type": 1, "min": int(min_age), "max": int(max_age) }
    elif "Trên" in age:
        min_age = age.replace("Trên ", "")
        return { "type": 4, "min": int(min_age)}
    elif "Dưới" in age:
        max_age = age.replace("Dưới ", "")
        return { "type": 3, "min": int(max_age)}  
    else:
        fixed = age.strip()
        return { "type": 2, "fixed": int(fixed)}  
    return { "type": 0 }

# Experience
def parse_experience(experience):
    if experience is None or experience == "Chưa có kinh nghiệm":
        return { "type": 0 }
    if "-" in experience:
        strip_experience = experience.replace("Năm", "").strip()
        min_exp, max_exp = strip_experience.split("-")
        return {"type": 1, "min": int(min_exp), "max": int(max_exp) }
    elif "Lên đến" in experience:
        strip_experience = experience.replace("Lên đến", "").replace("Năm", "").strip()
        max = strip_experience
        return { "type": 3, "max": int(max) }
    elif "Trên" in experience:
        strip_experience = experience.replace("Trên", "").replace("Năm", "").strip()
        min = strip_experience
        return { "type": 4, "min": int(min) }
    else:
        fixed = experience.replace("Năm", "").strip()
        return { "type": 2, "fixed": int(fixed)}
    return { "type": 0 }

# Expiration
df = df.withColumn("expiration", to_timestamp(concat(col("expiration"), lit(" 00:00:00")), "dd/MM/yyyy HH:mm:ss"))

# Update at
df = df.withColumn("update_time", to_timestamp(concat(col("update_time"), lit(" 00:00:00")), "dd/MM/yyyy HH:mm:ss"))

# Type
def get_type_job(type):
    if type == "Nhân viên chính thức":
        return 0
    elif type == "Bán thời gian":
        return 2
    elif type == "Thời vụ/ Nghề tự do":
        return 1
    elif type == "Thực tập":
        return 4
    else:
        return 5
def get_list_type_job(types):
    # Thời vụ/ Nghề tự do, Thực tập
    # type = types.split(",")
    if types is None:
        return []
    return [get_type_job(word.strip()) for word in types.split(",")]


classify_type_udf = udf(get_list_type_job, ArrayType(IntegerType()))
df = df.withColumn("type", classify_type_udf(col("type")))

# Role
def classify_role(role):
    if role is None:
        return 0
    role = role.strip()
    if role == "Nhân viên":
        return 0
    elif role == "Quản lý" or role == "Quản lý cấp trung":
        return 1
    elif role == "Cộng tác viên":
        return 6
    elif role == "Trưởng nhóm / Giám sát":
        return 5
    elif role == "Sinh viên/ Thực tập sinh" or role == "Mới tốt nghiệp":
        return 4
    elif role == "Giám đốc" or role == "Tổng giám đốc":
        return 2
    elif role == "Phó Giám đốc":
        return 3
    elif role == "Chuyên gia":
        return 7
    else:
        return 0
classify_role_udf = udf(classify_role, IntegerType())
df = df.withColumn("role", classify_role_udf(col("role")))

def normalize_benefit(benefits):
    return list(map(lambda benefit: re.sub(r'^[*\-+]+', '', benefit).strip(), benefits))
benefit_udf = udf(normalize_benefit, ArrayType(StringType()))
df = df.withColumn("benefit", benefit_udf(col("benefit")))
df = df.withColumn("requirement", benefit_udf(col("requirement")))
df = df.withColumn("description", benefit_udf(col("requirement")))

# location
def normalize_location(locations):
    arr = []
    for location in locations:
        parts = [part.strip() for part in location.split(',') if part.strip()]
        if len(parts) >= 2:
            province = parts[-1]
            district = parts[-2]
            address = ', '.join(parts[:-2])
            arr.append({"province": province, "district": district, "address": address})
        elif len(parts) == 1:
            arr.append ({"province": parts[-1], "district": None, "address": None })
        else: 
            arr.append({"address": None, "district": None, "province": None})
    return arr

rdd = df.rdd.map(lambda row: {
    **row.asDict(),
    "salary": parse_salary(row["salary"]),
    "age": parse_age(row["age"]),
    # "location": normalize_location(row["location"]),
    "experience": parse_experience(row["experience"])
})

# df.select(["role", "url"]).show(30, False)

# # Insert dữ liệu vào collection "jobs"
# rdd.foreachPartition(insert_into_mongodb)